In [251]:
import pandas as pd
import urllib
from bs4 import BeautifulSoup

In [252]:
def func_get_text(cell):
    return cell.get_text(strip=True)

In [253]:
def process_radiation_search(download_path):
    html_file = open(download_path, "r")
    soup = BeautifulSoup(html_file.read(), 'lxml')
    table = soup.find_all("table")[0]
    gammas = table.find_all("tr")
    init_dict = {"E_tab": [0], "Ig": [0], "Decay_Mode": ["b-"], "half_life": ["3 m"], "Isotope": ["Tu"]}
    gamma_df = pd.DataFrame(init_dict)

    for gamma_row in gammas[2:-2]:
        vals = list(map(func_get_text, gamma_row.find_all("td")))
        print(vals)
        gamma_df = gamma_df.append({"E_tab": vals[0], "Ig": vals[1], "Decay_Mode": vals[2], "half_life": vals[3], "Isotope": vals[4]}, ignore_index=True)
    return gamma_df

In [254]:
def find_gammas(energy, fwhm, t_min, t_max):
    e_min = energy - fwhm
    e_max = energy + fwhm
    request_url = f"http://nucleardata.nuclear.lu.se/toi/Gamma.asp?sql=&Min={e_min}&Max={e_max}&HlifeMin={t_min}&tMinStr={t_min}+s&HlifeMax={t_max}&tMaxStr={t_max}+s"
    print(request_url)
    output_file = f"downloads/searchE{energy}pm{fwhm}tMin{t_min}tMax{t_max}.html"
    urllib.request.urlretrieve (request_url, output_file)
    print(f"File {output_file} downloaded.")
    gamma_df = process_radiation_search(output_file)
    gamma_df["Energy"] = energy
    gamma_df["FWHM"] = fwhm
    gamma_df["half_life_searched"] = f"{t_min} - {t_max}"
    gamma_df = gamma_df.drop([0])
    return gamma_df

In [255]:
def add_all_gammas(df, t_min, t_max):
    for row in df.iterrows():
        possible_gammas = find_gammas(row[1]["Energy"], row[1]["FWHM"], t_min, t_max)
        df = df.append(possible_gammas, ignore_index=True)
    print(f"DF size is {df.shape}")
    drop_columns = ["Fit", "Filename", "Sample Identification", "Sample Type", "Sample Geometry"]
    drop_columns2 = ["Sample Size", "Efficiency ID", "Peak Analysis Report                    2.12.2020  10"]
    drop_columns3 = ["Peak Analysis From Channel", "Peak Search Sensitivity", "Max Iterations"]
    drop_columns4 = ["Use Fixed FWHM", "Peak Fit Engine Name"]
    df = df.drop(columns=drop_columns)
    df = df.drop(columns=drop_columns2)
    df = df.drop(columns=drop_columns3)
    df = df.drop(columns=drop_columns4)
    print(df.columns)
    return df

In [256]:
df = pd.read_csv("parsed_reports/M1-C1-Udepl-120.csv", index_col=0)
all_gammas = add_all_gammas(df, 120, 1e5)

all_gammas.to_csv("out/M1-C1-Udepl-120.csv", index=False)


['1078.44', '0.263', 'b-', '18.1\xa0m1', '159Eu']
['1078.53', '0.099', 'e+b+', '18.0\xa0m2', '109Sn']
['1078.67', '0.154', 'e+b+', '7.3\xa0m1', '103Cd']
['1078.64', '21', 'b-', '9.01\xa0h3', '128Sb']
['1078.6210', '0.56419', 'b-', '60.55\xa0m6', '212Bi']
['1078.71', '11.53', 'b-', '10.6\xa0m2', '142Ba']
['1078.763', '0.0637', 'b-', '1.728\xa0h1', '149Nd']
http://nucleardata.nuclear.lu.se/toi/Gamma.asp?sql=&Min=1204.17&Max=1204.65&HlifeMin=120&tMinStr=120+s&HlifeMax=100000.0&tMaxStr=100000.0+s
File downloads/searchE1204.41pm0.24tMin120tMax100000.0.html downloaded.
['1204.22', '0.226', 'b-', '55.4\xa0m4', '133mTe']
['1204.20812', '7.6218', 'b-', '8.12\xa0m12', '74Ga']
['1204.31', '14.2323', 'b-', '10.6\xa0m2', '142Ba']
['1204.3129', '0.093', 'e+b+', '38.5\xa0m15', '137Nd']
['1204.465', '0.029620', 'e+b+', '20.8\xa0h1', '100Rh']
['1204.54', '0.03513', 'b-', '14.08\xa0m8', '138Xe']
['1204.61', '1.33', 'b-', '2.68\xa0m7', '154mPm']
['1204.6013', '1.6317*', 'e+b+', '8.84\xa0m6', '188Au']
ht